In [ ]:
# prep for mount
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

#!python -m pip install wandb -Uq
#!python -m pip install ray[tune]
##!python -m pip install sigopt
#!python -m pip install optuna
!python -m pip install nltk python-Levenshtein
#!python -m pip install rouge-score
!python -m pip install peft

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader, random_split
import time
import logging
import numpy as np
#import wandb
import random
import math
import os
import pandas as pd
import re
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein as lev
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
#from rouge_score import rouge_scorer
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers.models.gpt2.modeling_gpt2 import GPT2Attention
from transformers import GPT2Config,  AutoModelForSeq2SeqLM
from peft import get_peft_model, IA3Config, TaskType

In [ ]:
#os.environ["WANDB_API_KEY"] = "a5d8e3b6d2ef7d55f930ab72670aaa64e1a4198d"

#wandb.login() #enter this token: a5d8e3b6d2ef7d55f930ab72670aaa64e1a4198d
#wandb.init(project="huggingface", entity="teamproject464-universit-t-des-saarlandes-saarland-unive")

In [ ]:
!ls '/content/drive/MyDrive'

 annotation_met_new_version.csv   eng_lit_mihan.csv	      gpt-small-fr-finetuned
 annotation_nonmet.csv		  eng_met_mihan.csv	      gpt-small-frfr-finetuned
'Colab Notebooks'		  french_gpt_training.ipynb   lit_met.txt
'Copy of gpt_training.ipynb'	  german_gpt_training.ipynb   results
 df_met_final.txt		  gpt-finetuned
 eng_all_viet.csv		  gpt-small-eng-finetuned


In [ ]:

# Preprocessing english data

def clean_output(text):
    # Remove commas, parentheses, and square brackets
    return re.sub(r'[\(\)\[\]\']', '', text)

def clean_output_special(text):
    if pd.isna(text):
        return text
    return re.sub(r'[\[\]\']' , '', text)
df_literal = pd.read_csv('/content/drive/MyDrive/eng_lit_mihan.csv', delimiter=';', encoding='ISO-8859-1')


df_literal['Output'] = df_literal['Output'].apply(clean_output_special)
df_literal['Input'] = df_literal['Input'].str.strip()
df_literal['Output'] = df_literal['Output'].str.strip()
df_literal = df_literal.dropna(subset=['Input', 'Output'])
# size 350
# for index, row in df_cleaned.iterrows():
#     print(index)
#     print(row['Input'])
#     print(row['Output'])


# Output (Subject, Verb, Object For Active voices) and (Object, Verb, Subjet For Passive Voices)
df_met = pd.read_csv('/content/drive/MyDrive/eng_met_mihan.csv', delimiter=';', encoding='UTF-8')

df_met = df_met.rename(columns={'Output (Subject, Verb, Object For Active voices) and (Object, Verb, Subjet For Passive Voices)': 'Output'})
df_met['Output'] = df_met['Output'].apply(clean_output)
df_met['Input'] = df_met['Input'].str.strip()
df_met['Output'] = df_met['Output'].str.strip()
df_met = df_met.drop(columns=['Verb'])
df_met = df_met.dropna(subset=['Input', 'Output'])

# size 350
# Now Viet's
eng_all_viet = pd.read_csv('/content/drive/MyDrive/eng_all_viet.csv',delimiter=';', encoding='UTF-8' )
eng_all_viet = eng_all_viet.rename(columns={'Tuple SVO': 'Output'})
eng_all_viet = eng_all_viet.rename(columns={'Sentence': 'Input'})

# swap the columns to match the other dataset
cols = list(eng_all_viet.columns)

# Swap the first two elements
#cols[0], cols[1] = cols[1], cols[0]

# Reorder the DataFrame using the new column order
# why 399 ? 235 literal and 164 metaphores ? sure
#eng_all_viet = eng_all_viet[cols]

eng_all_viet['Output'] = eng_all_viet['Output'].apply(clean_output)
empty_count = 0
met_count = 0

for index, row in eng_all_viet.iterrows():
    if row['Output'] == '':
        empty_count += 1
    else:
        met_count += 1


eng_df_all = pd.concat([eng_all_viet, df_met, df_literal], ignore_index=True)


eng_df_all = eng_df_all.dropna()

# Put a placeholder for all the instances where there is no metaphor, as NAN cannot be procesed later
eng_df_all['Output'] = eng_df_all['Output'].replace("", " ")
# pad the ouptuts and ensure there is always a triple
def ensure_triple(data):
    result = []
    for item in data:
        #item = item.replace(",", "|")
        item_list = [x.strip() for x in item.split(",")]
        # If the item is a tuple or list, convert it to a list and check its length
        if len(item_list) < 3:
            item_list.append('')
            # If it has less than 3 elements, add 'nothing' to fill the missing slots
            while len(item_list) < 3:
                item_list.append("")

        item =" ".join(item_list)
        result.append(item)
    return result

eng_df_all['Output'] = ensure_triple(eng_df_all['Output'])
eng_df_all

,Input,Output
0,"Mr. Reagan, in his first term, tried to kill t...",Mr. Reagan kill agency
1,The message was programmed to be seen only onc...,message kill itself
2,But let 's be specific and kill off only those...,we kill off program
3,"Centralization , on the other hand , kills mot...",Centralization kill motivation
4,"Making the Paris-Dakar rally less perilous , M...",Making Paris-Dakar rally kill it
...,...,...
864,You have to build up the equipment to a certai...,
865,Heavy water is used to control and cool the re...,
866,Engineers in uniforms and hardhats are everywh...,
867,Mr. Bilirakis's letter referred to the March 2...,


In [ ]:
# Optimizersss
#1. Optuna

def optuna_hp_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
    }

#2. SigOpt

def sigopt_hp_space(trial):

    return [
        {"bounds": {"min": 1e-6, "max": 1e-4}, "name": "learning_rate", "type": "double"},
        {
            "categorical_values": ["16", "32", "64", "128"],
            "name": "per_device_train_batch_size",
            "type": "categorical",
        },
    ]

#3. raytune

def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([16, 32, 64, 128]),
    }

#4.Wandb
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [16, 32, 64, 128]},
        },
    }

In [ ]:
# # include new metrics in class TRAINER

# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         # Get model predictions
#         outputs = model(**inputs)
#         logits = outputs.get('logits')
#         labels = inputs.get('labels')

#         # Convert logits to predicted tokens
#         predictions = torch.argmax(logits, dim=-1)

#         # Compute standard cross-entropy loss (you can keep this for token-level guidance)
#         loss_fct = torch.nn.CrossEntropyLoss()
#         loss = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

#         # Convert predictions and labels to text (for custom metrics)
#         pred_texts = [tokenizer.decode(pred, skip_special_tokens=True).strip() for pred in predictions]
#         label_texts = [tokenizer.decode(label, skip_special_tokens=True).strip() for label in labels]

#         # 1. Cosine Similarity Metric
#         cos_loss = 0.0
#         for pred_text, label_text in zip(pred_texts, label_texts):
#             if pred_text and label_text: # check that pred and label are non-empty
#               pred_vec = tokenizer.encode(pred_text, return_tensors='pt')
#               label_vec = tokenizer.encode(label_text, return_tensors='pt')
#               if pred_vec.shape[1] > 0 and label_vec.shape[1] > 0: # check that arrays are non-empty
#                 cos_sim = cosine_similarity(pred_vec.detach().numpy(), label_vec.detach().numpy())
#                 cos_loss += 1 - cos_sim.mean()  # Convert similarity to distance
#               else:
#                 # Handle cases where vectors are empty, possibly by assigning a default loss
#                 cos_loss += 1 # Assign highest possible loss (1) if vectors are empty
#             else:
#               cos_loss += 1 #Assign highest possible loss if predictions or labels are empty

#         # Normalize the cosine loss by batch size
#         cos_loss /= len(pred_texts)

#         # 2. Levenshtein Distance Metric
#         lev_loss = 0.0
#         for pred_text, label_text in zip(pred_texts, label_texts):
#             lev_distance = Levenshtein.distance(pred_text, label_text)
#             max_len = max(len(pred_text), len(label_text))
#             lev_loss += lev_distance / max_len  # Normalize Levenshtein distance by the max length

#         # Normalize Levenshtein loss by batch size
#         lev_loss /= len(pred_texts)

#         # Combine the original cross-entropy loss with custom metrics
#         combined_loss = loss + cos_loss + lev_loss

#         return (combined_loss, outputs) if return_outputs else combined_loss

In [ ]:
# import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:

def levenshtein_metric(predictions, references):
    distances = []
    for pred, ref in zip(predictions, references):
        pred_text = tokenizer.decode(pred, skip_special_tokens=True)
        ref_text = tokenizer.decode(ref, skip_special_tokens=True)
        # Compute the normalized Levenshtein distance
        distance = lev.distance(pred_text, ref_text) / max(len(pred_text), len(ref_text))
        distances.append(1 - distance)  # Invert it to represent similarity
    return np.mean(distances)

In [ ]:
def token_level_metrics(predictions, references):
    precision_list, recall_list, f1_list = [], [], []
    for pred, ref in zip(predictions, references):
        pred_ids = pred.flatten()  # Flatten the tensor if needed
        ref_ids = ref.flatten()

        # Ignore padding tokens (or other special tokens if defined)
        mask = ref_ids != tokenizer.pad_token_id

        # Compute precision, recall, f1
        precision = precision_score(ref_ids[mask], pred_ids[mask], average='micro')
        recall = recall_score(ref_ids[mask], pred_ids[mask], average='micro')
        f1 = f1_score(ref_ids[mask], pred_ids[mask], average='micro')

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return np.mean(precision_list), np.mean(recall_list), np.mean(f1_list)

In [ ]:
nltk.download('punkt')

# def bleu_score_metric(predictions, references):
#     bleu_scores = []
#     for pred, ref in zip(predictions, references):
#         pred_tokens = tokenizer.decode(pred, skip_special_tokens=True).split()
#         ref_tokens = tokenizer.decode(ref, skip_special_tokens=True).split()

#         # Calculate the BLEU score for each pair of sentences
#         bleu_scores.append(sentence_bleu([ref_tokens], pred_tokens))

#     return np.mean(bleu_scores)
def bleu_score_metric(predictions, references):
    bleu_scores = []
    smoothing_function = SmoothingFunction().method4  # Apply smoothing to handle missing n-grams
    for pred, ref in zip(predictions, references):
        pred_tokens = tokenizer.decode(pred, skip_special_tokens=True).split()
        ref_tokens = tokenizer.decode(ref, skip_special_tokens=True).split()

        # Calculate the BLEU score for each pair of sentences with smoothing
        bleu_scores.append(sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothing_function))

    return np.mean(bleu_scores)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

def calculate_rouge(reference, hypothesis):
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Compute ROUGE scores
    scores = scorer.score(reference, hypothesis)
    return scores

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    levenshtein_sim = levenshtein_metric(predictions, labels)
    precision, recall, f1 = token_level_metrics(predictions, labels)
    # bleu_score = bleu_score_metric(predictions, labels)
    #for Rouge
    # Convert predictions and labels to text
    # pred_texts = [tokenizer.decode(pred, skip_special_tokens=True).strip() for pred in predictions]
    # label_texts = [tokenizer.decode(label, skip_special_tokens=True).strip() for label in labels]
    # rouge_scores = [calculate_rouge(ref, pred) for ref, pred in zip(label_texts, pred_texts)]
    # rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
    # rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
    # rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])
    return {
        'levenshtein_similarity': levenshtein_sim,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        # 'bleu_score': bleu_score,
        # 'rouge1': rouge1,
        # 'rouge2': rouge2,
        # 'rougeL': rougeL
    }

In [ ]:
# # Hypertune with IA3
# class IA3Layer(nn.Module):
#     def __init__(self, hidden_dim):
#         super(IA3Layer, self).__init__()
#         # Learnable scaling vectors for keys, queries, and values
#         self.ia3_key = nn.Parameter(torch.ones(hidden_dim))
#         self.ia3_query = nn.Parameter(torch.ones(hidden_dim))
#         self.ia3_value = nn.Parameter(torch.ones(hidden_dim))

#     def forward(self, query, key, value):
#         # Apply input-aware scaling to keys, queries, and values

#         print(f"Query shape before IA3: {query.shape}", flush=True)
#         print(f"Key shape before IA3: {key.shape}",  flush=True)
#         print(f"Value shape before IA3: {value.shape}",  flush=True)

#         query = query * self.ia3_query
#         key = key * self.ia3_key
#         value = value * self.ia3_value

#         print(f"Query shape after IA3: {query.shape}",  flush=True)
#         print(f"Key shape after IA3: {key.shape}",  flush=True)
#         print(f"Value shape after IA3: {value.shape}",  flush=True)
#         return query, key, value

In [ ]:
# class GPT2AttentionWithIA3(GPT2Attention):
#     def __init__(self, config):
#         super().__init__(config)
#         # Initialize IA3 scaling layers
#         self.ia3 = IA3Layer(config.hidden_size)

#     def forward(self, hidden_states, layer_past=None, attention_mask=None, head_mask=None, use_cache=False, output_attentions=False):
#         # Perform the standard GPT-2 attention mechanism
#         print("1",  flush=True)
#         query, key, value = self._attn(hidden_states, attention_mask, head_mask, use_cache, output_attentions)
#         print("2",  flush=True)
#         # Apply IA3 scaling
#         query, key, value = self.ia3(query, key, value)
#         print("3",  flush=True)
#         # Proceed with the rest of the attention mechanism (unchanged)
#         return super().forward(hidden_states, layer_past, attention_mask, head_mask, use_cache, output_attentions)

In [ ]:
# class GPT2WithIA3(GPT2LMHeadModel):
#     def __init__(self, config):
#         super().__init__(config)
#         # Replace the attention layers with IA3-enhanced layers
#         for layer in self.transformer.h:
#             layer.attn = GPT2AttentionWithIA3(config)

# # Load the pre-trained GPT-2 model and replace the attention layers
# config = GPT2Config.from_pretrained("gpt2")
# model = GPT2WithIA3(config)

In [ ]:
model_name='gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
#config = GPT2Config.from_pretrained("gpt2")
#model = GPT2WithIA3(config)

# Add the padding token to GPT-2's tokenizer (optional, but useful)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Hook to capture layer activations
def get_layer_weights_hook(module, input, output):
    # Save the output (weights after decoding) from the layer
    module.layer_outputs = output.detach().cpu().numpy()

# Attach the hook to a specific layer you want to probe (e.g., the decoder layer)
model.lm_head.register_forward_hook(get_layer_weights_hook)
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=32, max_length_out = 32):
        self.inputs = df['Input'].tolist()
        self.outputs = df['Output'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_length_out= max_length_out

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]

        tokenized_output = self.tokenizer(output_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length_out)
        tokenized_input = self.tokenizer(input_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length)
        #print(tokenized_input['input_ids'].shape, tokenized_output['input_ids'].shape)

        # create Mask for empty and delimiter
        empty_token_id = self.tokenizer.convert_tokens_to_ids('<EMPTY>')
        output_mask = (output_ids != empty_token_id).long()
        # resize the model after introducing special tokens # and |
        model.resize_token_embeddings(len(tokenizer))
        # Return input_ids and attention_mask for training, no labels

        # Input IDs and attention mask
        input_ids = tokenized_input['input_ids'].squeeze()  # shape: (max_length)
        attention_mask = tokenized_input['attention_mask'].squeeze()  # shape: (max_length)
        output_ids = tokenized_output['input_ids'].squeeze()  # shape: (max_length)

        # Return input_ids and attention_mask for training, no labels
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': output_ids,
            'output_mask': output_mask
        }

class MaskedTextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=32, max_length_out = 32):
        self.inputs = df['Input'].tolist()
        self.outputs = df['Output'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_length_out= max_length_out
        self.logged_input_ids = []
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]

        tokenized_output = self.tokenizer(output_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length)
        tokenized_input = self.tokenizer(input_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length)
        #print(tokenized_input['input_ids'].shape, tokenized_output['input_ids'].shape)
        # Input IDs and attention mask
        input_ids = tokenized_input['input_ids'].squeeze()  # shape: (max_length)
        attention_mask = tokenized_input['attention_mask'].squeeze()  # shape: (max_length)
        output_ids = tokenized_output['input_ids'].squeeze()  # shape: (max_length)

        self.logged_input_ids.append(input_ids.tolist())
        # create Mask for empty and delimiter
        #hash_token_id = self.tokenizer.convert_tokens_to_ids('#')
        #delimiter_token_id = self.tokenizer.convert_tokens_to_ids('|')

        #output_mask = (output_ids != hash_token_id).long() & (output_ids != delimiter_token_id).long()
        # Return input_ids and attention_mask for training, no labels
        print(input_ids)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': output_ids
        }
    def get_input_ids(self):
        return self.logged_input_ids

class TextDatasetWithPrompts(Dataset):
    def __init__(self, df, tokenizer, max_length=64, max_length_out=64):
        self.inputs = df['Input'].tolist()
        self.outputs = df['Output'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_length_out = max_length_out

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = f"Input: {self.inputs[idx]}\nOutput:"
        output_text = self.outputs[idx]

        tokenized_input = self.tokenizer(
            input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt'
        )
        tokenized_output = self.tokenizer(
            output_text, truncation=True, padding='max_length', max_length=self.max_length_out, return_tensors='pt'
        )

        input_ids = tokenized_input['input_ids'].squeeze()
        attention_mask = tokenized_input['attention_mask'].squeeze()
        output_ids = tokenized_output['input_ids'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': output_ids
        }

# Load data into the custom dataset
My_dataset = MaskedTextDataset(df=eng_df_all, tokenizer=tokenizer)
# Split into training and test sets
split = 0.8
train_eval_size = int(split * len(My_dataset))
test_size = len(My_dataset) - train_eval_size
train_eval_data, test_data = random_split(My_dataset, [train_eval_size, test_size])

train_size = int(split * len(train_eval_data))
eval_size = len(train_eval_data) - train_size
training_data, eval_data = random_split(train_eval_data, [train_size, eval_size])

# Define DataLoaders
batch_size = 64
epochs = 50
# dataloader_train = DataLoader(training_data, batch_size=batch_size, shuffle=True)
# dataloader_eval = DataLoader(eval_data, batch_size=batch_size, shuffle=True)
# dataloader_test = DataLoader(test_data, batch_size=batch_size, shuffle=True)
######
# masking outside
#special_tokens_dict = {'additional_special_tokens': ['#']}
#tokenizer.add_special_tokens(special_tokens_dict)
#model.resize_token_embeddings(len(tokenizer))

######
# Define training arguments
training_args = TrainingArguments(
    logging_steps=50,
    output_dir='./results-small',
    learning_rate = 5e-4,
    weight_decay = 0.01,
    gradient_accumulation_steps=3,
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy="epoch",
    report_to="none"
)

# Model Init for hyperparam optimization


# model_args = {
#     "model": model_name,
#     "from_tf": False,
#     "config": AutoConfig.from_pretrained(model_name),
#     "cache_dir": None, # You can specify a cache directory if needed
#     "revision": None, # You can specify a model revision if needed
#     "token": True # Set to True if using an authentication token
# }

# def model_init(trial):
#     model = GPT2LMHeadModel.from_pretrained(
#         model_name,
#         ignore_mismatched_sizes=True,
#         from_tf=bool(".ckpt" in model_name),
#         config= AutoConfig.from_pretrained(model_name),
#         cache_dir=None,
#         revision=None,
#         token=True,
#     )
#     model.config.pad_token_id = tokenizer.eos_token_id
#     return model

#IA3 init config
peft_config = IA3Config(
    task_type=TaskType.CAUSAL_LM,  # Task type (change if necessary)
    target_modules=["attn.c_attn","attn.c_proj", "mlp.c_proj","mlp.c_fc"],  # Attention block layers to apply IA3
    feedforward_modules=["mlp.c_fc"],  # Apply to feedforward layers
    modules_to_save=["lm_head"],  # List of trainable modules to save
)
print(model)
# Wrap the model using the PEFT IA3 method
model = get_peft_model(model, peft_config)

# Optional: Print trainable parameters
model.print_trainable_parameters()

#Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
trainable params: 38,652,672 || all params: 163,092,480 || trainable%: 23.6998


/usr/local/lib/python3.10/dist-packages/peft/tuners/ia3/model.py:134: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# #Initialize the Trainer with wandb stuff
# trainer = Trainer(
#     model_init=model_init, # This is for the model init for the hyperparm optim
#     model=model,
#     args=training_args,
#     train_dataset=training_data,
#     eval_dataset=eval_data
# )
# trainer = CustomTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=training_data,
#     eval_dataset=eval_data,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics  # Optionally include custom metrics
# )

In [ ]:
training_data.get_input_ids()

AttributeError: 'Subset' object has no attribute 'get_input_ids'

In [ ]:
#
# ## Get best hyperparams
# best_trials = trainer.hyperparameter_search(
#     direction=["minimize", "maximize"],
#     backend="optuna",
#     hp_space=optuna_hp_space,
#     n_trials=10
# )

In [ ]:
# Start finetuning
start_time = time.time()
print('Training starts')
trainer.train()
model.save_pretrained("/content/drive/MyDrive/gpt-small-eng-finetuned")
#model.save_weights("/content/drive/MyDrive/weights.h5")
print('Training done')
end_time = time.time()

Training starts
tensor([  317,  1256,   286,   262, 15855,  7713,   326,   423,  6825,   262,
         8848,  1244,   765,   284,  1011,   262,  3663,   284,   651,   287,
          783,   837, 10148,   339,   531,   764, 50256, 50256, 50256, 50256,
        50256, 50256])
tensor([   50, 44262,  2741, 24070,   517, 26016, 24173,   621,  5085,   286,
          597,   584,   471,    13,    50,    13,  3814,   938,   614,    11,
          290, 12503,   351, 18696,  2029,   720,  2327,    11,   830, 24070,
          517,   621])
tensor([25378,   373,   788,  3888,   287,   584,  3453,   864, 11678,    11,
          475,   262,  7664,   286,   465,  1807,   373,  1598,    25,   632,
          338,  5802,   284,  1445,  3049,   618,   345,   821, 25046,   257,
         5509,    13])
tensor([ 6653,  1171,    12, 39468,   661,   423, 21050,  7638,   351,  9088,
          286,   465, 18138,    13, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 

Epoch,Training Loss,Validation Loss


tensor([ 4711, 10150,   389,  1336,   286,   262,  5916,    11,   285,   692,
          385,   591,    11,   290,  1402, 20048,   558,   504,   326,  6804,
         5937,   588,   284,  4483,    13, 50256, 50256, 50256, 50256, 50256,
        50256, 50256])
tensor([  464, 26119,  7603,  8295, 33596,   645,  2392,  3252,   326, 11104,
          481,  2222,   257, 16457,   393,   326,  1393,  2494,  5732,   481,
         3608,  7172,  4581,    13, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256])
tensor([16678, 33124,   837,   543,   389,  5207,   286,  7446,   326,  4859,
          284,   262,  7446,   287,   257,  6291,   837,   423,  2810,  8780,
         2370,   287,   517,   621,   257,  8667,  5123,   290,  7262,  2663,
         1141,   262])
tensor([ 4342,   287, 11066, 11729,    11,  5941,   286,   257,  1448,  1900,
          355,   262, 49673,   286, 14611, 16999,   364,   547, 30040,  7384,
          416, 14830,   292,   287,   257,   880,    12, 47045,  4519,   

NameError: name 'rouge_scorer' is not defined

In [ ]:
fine_tuning_time = end_time - start_time
print(f"Training completed in {fine_tuning_time:.2f} seconds.")
trainer.evaluate()

NameError: name 'end_time' is not defined

In [ ]:
predictions, labels, metrics = trainer.predict(test_dataset=test_data)

Epoch,Training Loss,Validation Loss,Levenshtein Similarity,Precision,Recall,F1,Bleu Score,Rouge1,Rouge2,Rougel
1,No log,8.614807,0.057543,0.004206,0.004206,0.004206,0.004124,0.040961,0.003553,0.038240
2,No log,5.652208,0.064249,0.006364,0.006364,0.006364,0.004618,0.043603,0.003675,0.041546
3,No log,3.312723,0.089761,0.006364,0.006364,0.006364,0.004582,0.047292,0.001859,0.042668
4,No log,1.830143,0.124767,0.002767,0.002767,0.002767,0.003902,0.040689,0.000000,0.036155
5,No log,1.101771,0.110878,0.001660,0.001660,0.001660,0.000375,0.014206,0.000000,0.014206
6,No log,0.862203,0.062002,0.001660,0.001660,0.001660,0.000108,0.009661,0.000000,0.007605
7,No log,0.803494,0.075417,0.008301,0.008301,0.008301,0.000108,0.006852,0.000000,0.006852
8,No log,0.793743,0.097288,0.015495,0.015495,0.015495,0.000217,0.008907,0.000000,0.008907
9,3.013500,0.786064,0.083946,0.015495,0.015495,0.015495,0.000504,0.011648,0.000000,0.011648
10,3.013500,0.754237,0.093115,0.032928,0.032928,0.032928,0.000527,0.011391,0.000000,0.011391


In [ ]:
predicted_token_ids = np.argmax(predictions, axis=-1)

# Step 2: Decode the predicted token IDs to text
decoded_predictions = [tokenizer.decode(pred_seq, skip_special_tokens=True) for pred_seq in predicted_token_ids]
decoded_predictions

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  to',
 ' ',
 ' ',
 '  price',
 ' ',
 ' ',
 '  the',
 ' ',
 ' aryler',
 ' ',
 ' ',
 '  of',
 ' ',
 ' ',
 ' ',
 ' ',
 '  or',
 ' ',
 ' ',
 ' ',
 ' ',
 '  Shining Lie',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ract-',
 ' ',
 ' ',
 '  price target',
 ' ',
 ' ',
 ' ',
 ' - trader',
 ' ',
 '  lend',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  hand',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  its civilian',
 ' ',
 ' ',
 ' ',
 '  the',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  tail',
 '  the',
 ' ',
 ' ',
 '  Flight audience',
 ' ',
 '  and funds billion',
 ' ',
 ' ',
 ' ',
 ' ll',
 ' ',
 '  be',
 ' ',
 '  out',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  me',
 ' ',
 ' iege',
 ' ure',
 '  be',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  order',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  reality',
 ' ',
 ' ',
 '  England',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' er',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' '

In [ ]:
decoded_labels = [tokenizer.decode(label_seq, skip_special_tokens=True) for label_seq in labels]
decoded_labels

['government officials play a role',
 '  ',
 'motor-mouth examine the process',
 '  ',
 '  ',
 'Mr. Mitterrand dissolve the National Assembly',
 '  ',
 'the Bush campaign dissolve group',
 '  ',
 '  ',
 'Sunbelt roll up a $1.2 billion net loss',
 '  ',
 'improvement lend support',
 '  ',
 '  ',
 'CDs be rolled over ',
 '  ',
 'Employee leasing be killed Reform Act',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 'Johnson & Johnson stumble upon a Fountain of Youth Cream',
 'let us not pass over page',
 'those trim judgement',
 'the action smooth volatility',
 '  ',
 '  ',
 '  ',
 'the Association attack the restrictiveness',
 '  ',
 '  ',
 'stronger start lend hand',
 'the business be dragged down the stock price',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 'Court melt down statue',
 '  ',
 '  ',
 '  ',
 'mugger assault woman',
 'every scene miss something',
 'opponents ride roughshod',
 '  ',
 'the surplus melt away ',
 '  ',
 '  ',
 'he dis

In [ ]:
# human comparison:
for label,prediction in zip(decoded_labels,decoded_predictions):
  print(f'Label:{label}, Prediction:{prediction}')

Label:government officials play a role, Prediction: 
Label:  , Prediction: 
Label:motor-mouth examine the process, Prediction: 
Label:  , Prediction: 
Label:  , Prediction: 
Label:Mr. Mitterrand dissolve the National Assembly, Prediction:  to
Label:  , Prediction: 
Label:the Bush campaign dissolve group, Prediction: 
Label:  , Prediction:  price
Label:  , Prediction: 
Label:Sunbelt roll up a $1.2 billion net loss, Prediction: 
Label:  , Prediction:  the
Label:improvement lend support, Prediction: 
Label:  , Prediction: aryler
Label:  , Prediction: 
Label:CDs be rolled over , Prediction: 
Label:  , Prediction:  of
Label:Employee leasing be killed Reform Act, Prediction: 
Label:  , Prediction: 
Label:  , Prediction: 
Label:  , Prediction: 
Label:  , Prediction:  or
Label:  , Prediction: 
Label:  , Prediction: 
Label:Johnson & Johnson stumble upon a Fountain of Youth Cream, Prediction: 
Label:let us not pass over page, Prediction: 
Label:those trim judgement, Prediction:  Shining Lie
Labe

In [ ]:
!python -m pip install safetensors

In [ ]:
# Probing

from sklearn.decomposition import PCA
from safetensors.torch import load_file

# Load the weights
file_path = "adapter_model.safetensors"
weights = load_file("/content/drive/MyDrive/gpt-small-eng-finetuned/adapter_model.safetensors")





In [ ]:
#for every iteration I have weights
weights.keys()

dict_keys(['base_model.model.lm_head.weight', 'base_model.model.transformer.h.0.attn.c_attn.ia3_l', 'base_model.model.transformer.h.0.attn.c_proj.ia3_l', 'base_model.model.transformer.h.0.mlp.c_fc.ia3_l', 'base_model.model.transformer.h.0.mlp.c_proj.ia3_l', 'base_model.model.transformer.h.1.attn.c_attn.ia3_l', 'base_model.model.transformer.h.1.attn.c_proj.ia3_l', 'base_model.model.transformer.h.1.mlp.c_fc.ia3_l', 'base_model.model.transformer.h.1.mlp.c_proj.ia3_l', 'base_model.model.transformer.h.10.attn.c_attn.ia3_l', 'base_model.model.transformer.h.10.attn.c_proj.ia3_l', 'base_model.model.transformer.h.10.mlp.c_fc.ia3_l', 'base_model.model.transformer.h.10.mlp.c_proj.ia3_l', 'base_model.model.transformer.h.11.attn.c_attn.ia3_l', 'base_model.model.transformer.h.11.attn.c_proj.ia3_l', 'base_model.model.transformer.h.11.mlp.c_fc.ia3_l', 'base_model.model.transformer.h.11.mlp.c_proj.ia3_l', 'base_model.model.transformer.h.2.attn.c_attn.ia3_l', 'base_model.model.transformer.h.2.attn.c_pro

In [ ]:
for layer_name in weights.keys():
    print(f"Layer: {layer_name}, Shape: {weights[layer_name].shape}")


Layer: base_model.model.lm_head.weight, Shape: torch.Size([50257, 768])
Layer: base_model.model.transformer.h.0.attn.c_attn.ia3_l, Shape: torch.Size([2304, 1])
Layer: base_model.model.transformer.h.0.attn.c_proj.ia3_l, Shape: torch.Size([768, 1])
Layer: base_model.model.transformer.h.0.mlp.c_fc.ia3_l, Shape: torch.Size([1, 768])
Layer: base_model.model.transformer.h.0.mlp.c_proj.ia3_l, Shape: torch.Size([768, 1])
Layer: base_model.model.transformer.h.1.attn.c_attn.ia3_l, Shape: torch.Size([2304, 1])
Layer: base_model.model.transformer.h.1.attn.c_proj.ia3_l, Shape: torch.Size([768, 1])
Layer: base_model.model.transformer.h.1.mlp.c_fc.ia3_l, Shape: torch.Size([1, 768])
Layer: base_model.model.transformer.h.1.mlp.c_proj.ia3_l, Shape: torch.Size([768, 1])
Layer: base_model.model.transformer.h.10.attn.c_attn.ia3_l, Shape: torch.Size([2304, 1])
Layer: base_model.model.transformer.h.10.attn.c_proj.ia3_l, Shape: torch.Size([768, 1])
Layer: base_model.model.transformer.h.10.mlp.c_fc.ia3_l, Shap

In [ ]:
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Get embeddings for two specific tokens (e.g., "dog" and "cat")
token_ids = tokenizer.convert_tokens_to_ids(["dog", "cat"])
token_embeddings = weights["transformer.wte.weight"][token_ids].cpu().detach().numpy()

# Compute cosine similarity
similarity = cosine_similarity(token_embeddings[0].reshape(1, -1), token_embeddings[1].reshape(1, -1))
print(f"Cosine similarity between 'dog' and 'cat': {similarity}")